# Phi-4 Mini ONNX Паралельний Виклик Функцій: Посібник

Цей нотатник демонструє, як використовувати Phi-4 Mini з ONNX Runtime GenAI для паралельного виклику функцій. Виклик функцій дозволяє моделі розумно використовувати зовнішні інструменти та API відповідно до запитів користувача.

## Огляд

У цьому посібнику ви навчитеся:
- Налаштовувати Phi-4 Mini з ONNX Runtime GenAI
- Визначати схеми функцій для бронювання авіаквитків і готелів
- Використовувати керовану генерацію з граматикою Lark для структурованого виводу
- Виконувати паралельні виклики функцій для складних сценаріїв бронювання подорожей

## Попередні вимоги

Перед запуском цього нотатника переконайтеся, що ви:
- Завантажили модель Phi-4 Mini ONNX
- Встановили пакет `onnxruntime-genai`
- Маєте базове розуміння концепцій виклику функцій


## Крок 1: Імпорт необхідних бібліотек

Спочатку імпортуємо потрібні бібліотеки для реалізації виклику функцій.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Крок 2: Налаштування та конфігурація моделі

Тепер ми налаштуємо модель Phi-4 Mini ONNX. Переконайтеся, що ви замінили шлях на вашу фактичну директорію моделі.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Крок 3: Налаштуйте параметри генерації

Налаштуйте параметри генерації, щоб контролювати поведінку моделі при створенні відповідей. Ці налаштування забезпечують детерміновані та зосереджені відповіді для виклику функцій.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Крок 4: Визначення доступних функцій

Тут ми визначаємо функції, які наш AI-асистент може викликати. У цьому прикладі є дві функції, пов’язані з подорожами:
1. **booking_flight_tickets**: Для бронювання авіаквитків між аеропортами
2. **booking_hotels**: Для бронювання готельного проживання

Визначення функцій відповідають формату схеми виклику функцій OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Крок 5: Допоміжні функції для створення граматики

Ці допоміжні функції перетворюють наші визначення функцій у формат граматики Lark, який використовується ONNX Runtime GenAI для керованої генерації. Це гарантує, що модель створює коректні виклики функцій у правильному форматі JSON.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Крок 6: Тестування генерації граматики

Давайте протестуємо наші функції генерації граматики, щоб побачити, як вони перетворюють визначення інструментів у відповідний формат.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Крок 7: Підготовка системного запиту та генератора

Тепер ми створимо системний запит, який інформує модель про доступні інструменти, і налаштуємо генератор із параметрами керованої генерації.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Крок 8: Ініціалізація генератора з керованою генерацією

Тепер ми створимо генератор із налаштованими параметрами та застосуємо граматику Lark для керованої генерації.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Крок 9: Тестування паралельного виклику функцій

Тепер давайте протестуємо нашу конфігурацію на складному сценарії бронювання подорожі, який вимагає виклику кількох функцій.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Генерація викликів функцій

Модель тепер буде генерувати структуровані виклики функцій на основі запиту користувача. Завдяки керованій генерації, вихідні дані будуть у форматі JSON, який можна безпосередньо виконати.

**Очікуваний результат**: Модель повинна згенерувати виклики функцій для:
1. `booking_flight_tickets` з деталями рейсу з Пекіна (PEK) до Парижа (CDG)
2. `booking_hotels` з деталями проживання в Парижі

Запустіть комірку нижче, щоб побачити живу генерацію:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Висновок

🎉 **Вітаємо!** Ви успішно реалізували паралельний виклик функцій з Phi-4 Mini, використовуючи ONNX Runtime GenAI.

### Що ви дізналися:

1. **Налаштування моделі**: Як налаштувати Phi-4 Mini з ONNX Runtime GenAI
2. **Визначення функцій**: Як визначати схеми функцій для виклику функцій штучного інтелекту
3. **Спрямована генерація**: Як використовувати граматику Lark для створення структурованого виводу
4. **Паралельні виклики функцій**: Як обробляти складні запити, що потребують кількох викликів функцій

### Основні переваги:

- ✅ **Структурований вивід**: Спрямована генерація забезпечує коректні JSON-виклики функцій
- ✅ **Паралельна обробка**: Можливість обробляти кілька викликів функцій в одному запиті
- ✅ **Висока продуктивність**: ONNX Runtime забезпечує оптимізоване виконання
- ✅ **Гнучка схема**: Легко додавати або змінювати визначення функцій

### Ресурси:

- [Документація Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Документація ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Найкращі практики виклику функцій](https://platform.openai.com/docs/guides/function-calling)



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, зверніть увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ мовою оригіналу слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний переклад людиною. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
